In [1]:
# Kernel should be set to Python 2 to accomodate Multitracker

import multi_tracker_analysis as mta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
import glob, math, os
from __future__ import division

1.5.1
recommended version: 1.1.1 or greater


#### Usage notes for Multitracker
Multitracker requires ROS (roslib) to run. However, ros kinetic changes the python path during setup, which causes issues with running OpenCV (cv2). The Anaconda3 install on the lab Ubuntu computer currently doesn't run CV2 and ROS concurrently from the same environment. To switch between the two, open the /home/username/.bashrc file (use Ctrl-H to see hidden files in Ubuntu Files). Comment out the ros/kinetic/source lines when using CV2, and reopen a new Jupyter Notebook to allow the changes to go into effect. 

In [2]:
'''
Translate Multitracker hdf5 files into Pandas dataframes and save to CSV.
This script will only run the analyses for files that have not been analyzed yet. 
When all files have already been analyzed there will be no Multitracker output. 
'''

fnames = glob.glob('/home/eleanor/Documents/gcamp_analysis_files_finished/*/data/')
df_names = pd.read_csv('./static_data_all_GCAMP.csv')

for n in range(0, len(fnames)):
    fname = fnames[n]
    name = fname[54:-6]
    
    videoname = '/home/eleanor/Documents/gcamp_videos/' + name + '.avi'
    savename = './gcamp_video_csvs/'+ name +'.csv'
    
    # Do not overwrite video files that have already been made and reviewed
    if not os.path.isfile(savename):
        df, config = mta.read_hdf5_file_to_pandas.load_and_preprocess_data(fname)

        video_clip = VideoFileClip(videoname)
        frame = np.array(video_clip.get_frame(0).astype(float))
        video_width, video_height = len(frame[0]), len(frame)
        df['pixel_width'] = video_width
        df['pixel_height'] = video_height
        
        # Mark frames detected by Multitracker to differentiate from manually entered frames.
        df['manual_tracker_fix'] = False
        
        # remove default columns added by multitracker that are not relevant or not calibrated to the data
        del df['angle'], df['area'], df['time_epoch'], df['time_epoch_nsecs'], df['time_epoch_secs'], \
        df['speed'], df['velocity_x'], df['velocity_y']

        df.to_csv(savename, index=False)
        del video_clip.reader, video_clip
        
print('Script finished running!')

Script finished running!


In [3]:
'''
Multitracker will not register objects until they begin to move.
In some videos, larvae do not start to move until several seconds into the experiment. 
Each of these videos were manually inspected to confirm lack of movement in the initial frames.
Next, the tracker's position when the larva was first detected was propagated to 
the beginning of the movie. 
All manually corrected frames are marked with ['manual_tracker_fix'] == True.
'''

# Print all files with missing frames for manual inspection. 
fnames = sorted(glob.glob('./gcamp_video_csvs/*.csv'))[::-1] # reversed list for alphabetical sorting
error_fnames = []
count = 0

for name in fnames:
    df = pd.read_csv(name)
    missed = 1799 - len(df)
    if missed > 2: # first frame is always missing because of background subtraction. 
        error_fnames.append(name[19:])
        print name[19:], ':', 1799-len(df), 'frames missing'
        count += 1
        
print '---'
print count, 'files have errors, out of', len(fnames), 'total files'

---
0 files have errors, out of 402 total files


In [4]:
'''
If the video has been manually checked to see if it has a long period of 
no motion at the beginning, then we can automatically propagate the first frame
data point backwards to the beginning of the video. 
This script will only rewrite files with missing frames, so if it has already been run
for this dataset the output should rewrite 0 files. 
'''

checked = pd.read_csv('./manually_checked_beginning_pause.csv')
checked_fnames = checked['filename'].tolist()
count = 0

for name in checked_fnames: 
    name_readable = "./gcamp_video_csvs/"+name+".csv"
    df = pd.read_csv(name_readable)

    frames = df['frames'].tolist()
    fmin = min(frames)
    fmin_row = df[df['frames'] == fmin]
    assert len(fmin_row) == 1
    
    if fmin > 2:
        for n in range(2, fmin):
            fmin_copy = fmin_row.copy()
            fmin_copy.ix[0, "objid"] = "manual"
            fmin_copy.ix[0, "frames"] = n
            df = df.append(fmin_copy)
            
        df = df.sort_values(by="frames")
        df.to_csv(name_readable, index=False)
        count += 1
    
print('Finished analyzing '+str(len(checked_fnames))+' files!')
print('Rewrote '+str(count)+' files!')

Finished analyzing 142 files!
Rewrote 0 files!


In [5]:
'''
Add some rudimentary analyses such as instantaneous speed.
Calculations are saved in a CSV copy rather than rewriting the original.
'''

fnames = glob.glob('./gcamp_video_csvs/*.csv')

def hypotenuse(leg1, leg2):
    '''Calculate the hypotenuse given two side lengths'''
    hyp = np.sqrt(leg1**2 + leg2**2)
    return(hyp)

for n in range(0, len(fnames)):
    fname = fnames[n]
    name = fname[19:-4]
    
    readname = './gcamp_video_csvs/' + name + '.csv'
    savename = './gcamp_video_calculations/'+ name +'.csv'

    df = pd.read_csv(readname)

    # convert pixel position to mm using the cropped video size as reference
    df['pos_x_mm'] = 80*df['position_x']/df['pixel_width'] # arena is 80mm wide
    df['pos_y_mm'] = 30*df['position_y']/df['pixel_height'] # arena is 30mm high
    df['pos_x_mm_pre'] = df.pos_x_mm.shift(1)
    df['pos_y_mm_pre'] = df.pos_y_mm.shift(1)

    # sometimes the tracker guess goes slightly off screen but this is not physically possible
    # because the larva is contained within the arena bounds. 
    df['zeroed_x_mm'] = df['pos_x_mm'].clip(lower=0, upper=80)
    df['zeroed_y_mm'] = df['pos_y_mm'].clip(lower=0, upper=30)
    df['zeroed_x_mm_pre'] = df.zeroed_x_mm.shift(1)
    df['zeroed_y_mm_pre'] = df.zeroed_y_mm.shift(1)

    # speed calculations
    df['height'] = abs(df['zeroed_y_mm'] - df['zeroed_y_mm_pre'])
    df['width'] = abs(df['zeroed_x_mm'] - df['zeroed_x_mm_pre'])
    df['speed_EL'] = hypotenuse(df.height, df.width)

    # remove columns used to generate the final calculations because we no longer need them
    del df['zeroed_x_mm_pre'], df['zeroed_y_mm_pre'], df['height'], df['width']
    del df['pos_x_mm_pre'], df['pos_y_mm_pre']

    df.to_csv(savename, index=False)
    
print('Finished calculating all analyses!')

Finished calculating all analyses!
